In [1]:

import datetime
import os
import random
import time

import torch.nn.functional as F
from models.FNO import FNO2d, FNO3d
from models.Unet import UNet2d, UNet3d
# from models.GFNO_steerable import GFNO2d_steer
# from models.Unet import Unet_Rot, Unet_Rot_M, Unet_Rot_3D
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import imageio
import pandas as pd
from models.DNO import DNO

from utils25 import  LpLoss, nse, corr, critical_success_index

import scipy
import numpy as np
from timeit import default_timer
import argparse
from torch.utils.tensorboard import SummaryWriter
import torch
import h5py
import xarray as xr
from tqdm import tqdm
from openpyxl import load_workbook
#os.environ["CUDA_VISIBLE_DEVICES"] = "3"
torch.set_num_threads(1)
if torch.cuda.is_available():
    device = torch.device('cuda:3')


#else:
device = torch.device('cpu')
def get_eval_pred(model, x, strategy, T, times):

    if strategy == "oneshot":
        pred = model(x)
    else:

        for t in range(T):
            t1 = default_timer()
            im = model(x)
            times.append(default_timer() - t1)
            if t == 0:
                pred = im
            else:
                pred = torch.cat((pred, im), -2)
            if strategy == "markov":
                x = im
            else:
                x = torch.cat((x[..., 1:, :], im), dim=-2)

    return pred

In [2]:
import torch
torch.backends.mkldnn.enabled = False


In [3]:

################################################################
# configs
################################################################
start_time = 202107081000
zoom_factor=1
width=int(5459/24)
height=int(6679/24)
print(width,height)
from argparse import Namespace
# Save all settings in args
args = Namespace(
    results_path="/home/sirui/INNOMAUS/UrbanFloodCast/UrbanFloodCast/TL-DNO/save/",
    suffix="seed1",
    txt_suffix="Flood_DNO_Layers_oneset_seed1_t5",
    super_res=False,
    verbose=True,
    T=120,  # number of timesteps to predict
    ntrain=4,  # training sample size
    nvalid=1,  # valid sample size
    ntest=1,  # test sample size
    nsuper=None,
    seed=1,
    model_type='DNO',
    depth=4,
    modes=12,
    width=20,
    Gwidth=10,  # hidden dimension of equivariant layers if model_type=hybrid
    n_equiv=3,  # number of equivariant layers if model_type=hybrid
    reflection=False,  # symmetry group p4->p4m for data augmentation
    grid=None,  # ["symmetric", "cartesian", None]
    epochs=500,
    early_stopping=100,  # stop if validation error does not improve for successive epochs
    batch_size=1,
    learning_rate=0.001,
    step=False,  # use step scheduler
    gamma=0.5,  # gamma for step scheduler
    step_size=None,  # step size for step scheduler
    lmbda=0.0001,  # weight decay for adam
    strategy="oneshot",  # markov, recurrent, or oneshot
    time_pad=False,  # pad the time dimension for strategy=oneshot
    noise_std=0.00,  # amount of noise to inject for strategy=markov
    data_path="/home/sirui/INNOMAUS/wurzburg/pt/",
    root='/home/sirui/INNOMAUS/UrbanFloodCast/UrbanFloodCast/TL-DNO/save/',
)



assert args.model_type in ["FNO2d", "FNO2d_aug",
                           "FNO3d", "FNO3d_aug",
                           "UNet2d", "UNet3d", "DNO"], f"Invalid model type {args.model_type}"
assert args.strategy in ["teacher_forcing", "markov", "recurrent", "oneshot"], "Invalid training strategy"

torch.manual_seed(args.seed)
np.random.seed(args.seed)
if device.type == 'cuda':
    torch.cuda.manual_seed(args.seed)
random.seed(args.seed)

data_aug = "aug" in args.model_type

TRAIN_PATH = args.data_path

# FNO data specs
Sy = height
Sx = width
S = 64 # spatial res
S_super = 4 * S # super spatial res
T_in = 1 # number of input times
T = args.T
T_super = 4 * T # prediction temporal super res
d = 2 # spatial res
num_channels = 5
num_channels_y = 3

# adjust data specs based on model type and data path
threeD = args.model_type in ["FNO3d",
                             "Unet_Rot_3D", "DNO", "UNet3d"]
swe = False
rdb = False
grid_type = "cartesian"
if args.grid:
    grid_type = args.grid
    assert grid_type in ['symmetric', 'cartesian', 'None']


ntrain = args.ntrain # 1000
nvalid = args.nvalid
ntest = args.ntest # 200

time_modes = None
time1 = args.strategy == "oneshot" # perform convolutions in space-time
if time1 and not args.time_pad:
    time_modes = 5 if swe else 8 # 6 is based on T=10
elif time1 and swe:
    time_modes = 8

modes = args.modes
n_layer = args.depth
batch_size = args.batch_size

epochs = args.epochs # 500
learning_rate = args.learning_rate
scheduler_step = args.step_size
scheduler_gamma = args.gamma # for step scheduler

initial_step = 1 if args.strategy == "markov" else T_in





227 278


In [4]:
import torch
import os
import numpy as np

################################################################
# Dataset class
################################################################
class flood_data(torch.utils.data.Dataset):
    def __init__(self, path_root, T_in, T_out=None, train=True, strategy="markov", std=0.0):
        self.markov = strategy == "markov"
        self.teacher_forcing = strategy == "teacher_forcing"
        self.one_shot = strategy == "oneshot"
        self.path_root = path_root
        # self.data = data[..., :(T_in + T_out)] if self.one_shot else data[..., :(T_in + T_out), :]
        self.data = []
        pt_files = [name for name in os.listdir(self.path_root) if name.endswith('.pt')]
        n = len(pt_files)
        print('Number of .pt files:', n)
        self.data = [os.path.join(self.path_root, name) for name in pt_files]
        self.nt = T_in + T_out
        self.T_in = T_in
        self.T_out = T_out
        self.num_hist = 1 if self.markov else self.T_in
        self.train = train
        self.noise_std = std

    def log_transform(self, data, eps=1e-2):
        return torch.log(1 + data/eps)

    def __len__(self):
        if self.train:
            if self.markov:
                return len(self.data) * (self.nt - 1)
            if self.teacher_forcing:
                return len(self.data) * (self.nt - self.T_in)
        return len(self.data)
    def downsample_data(self, data, scale_factor=0.125):
        data = data.permute(2, 3, 0, 1)  # (3213, 2727, 25, 5) -> (5, 25, 3213, 2727)
        data = F.interpolate(data, scale_factor=scale_factor, mode="bilinear", align_corners=False)
        data = data.permute(2, 3, 0, 1)  
        return data

    def __getitem__(self, idx):
        if not self.train or not (self.markov or self.teacher_forcing): # full target: return all future steps
            pde_path = self.data[idx]
            # path_idx = os.path.join(path_root, str(idx) + ".pt")
            pde = torch.load(pde_path)
            pde = self.downsample_data(pde,scale_factor=1/zoom_factor)
            # pde = pde.permute(1, 2, 0, 3)
            if self.one_shot:
                x = pde[..., :self.T_in, :3]
                mask = (x[..., 0:1] == 0.0)
                x[..., 1:2][mask] = 0.0
                x[..., 2:3][mask] = 0.0
                x[..., :3] = torch.nan_to_num(x[..., :3], nan=0.0)
                x1 = x.unsqueeze(-3).repeat([1, 1, self.T_out, 1, 1])
                # x_n = x.numpy()
                # x_n = np.nan_to_num(x_n, nan=-99999)
                # x_n = np.ma.masked_array(x_n, mask=(x_n < -2000))
                # x = torch.from_numpy(x_n).float()
                # nan_mask = torch.isnan(x)
                # contains_nan = torch.any(nan_mask)
                # print('contains_nan', contains_nan)
                p = pde[..., self.T_in:(self.T_in + self.T_out), 3:4]
                x2 = self.log_transform(p) / 10.0
                x2 = torch.unsqueeze(x2, dim = -1)
                z = pde[..., self.T_in:(self.T_in + self.T_out), 4:5]
                max_z = z.max()
                z_value = max_z + 30.0
                z = torch.nan_to_num(z, nan=z_value)
                x3 = torch.nn.functional.normalize(z)
                x3 = torch.unsqueeze(x3, dim=-1)
                x = torch.cat((x1, x2), dim=-1)
                x = torch.cat((x, x3), dim=-1)
                # x = x.unsqueeze(-3).repeat([1, 1, self.T_out, 1, 1])
                y = pde[..., self.T_in:(self.T_in + self.T_out), :3]
                mask_y = (y[..., 0:1] == 0.0)
                y[..., 1:2][mask_y] = 0.0
                y[..., 2:3][mask_y] = 0.0
                mask_tensor = ~torch.isnan(y)
                y = torch.nan_to_num(y, nan=0.0)
            else:
                x = pde[..., (self.T_in - self.num_hist):self.T_in, :3]
                mask = (x[..., 0:1] == 0.0)
                x[..., 1:2][mask] = 0.0
                x[..., 2:3][mask] = 0.0
                x[..., :3] = torch.nan_to_num(x[..., :3], nan=0.0)
                x1 = x.unsqueeze(-3).repeat([1, 1, self.T_out, 1, 1])
                # x_n = x.numpy()
                # x_n = np.nan_to_num(x_n, nan=-99999)
                # x_n = np.ma.masked_array(x_n, mask=(x_n < -2000))
                # x = torch.from_numpy(x_n).float()
                # nan_mask = torch.isnan(x)
                # contains_nan = torch.any(nan_mask)
                # print('contains_nan', contains_nan)
                p = pde[..., self.T_in:(self.T_in + self.T_out), 3:4]
                x2 = self.log_transform(p) / 10.0
                z = pde[..., self.T_in:(self.T_in + self.T_out), 4:5]
                max_z = z.max()
                z_value = max_z + 30.0
                z = torch.nan_to_num(z, nan=z_value)
                x3 = torch.nn.functional.normalize(z)
                x = torch.cat((x1, x2), dim=-1)
                x = torch.cat((x, x3), dim=-1)
                # x[..., :4] = self.log_transform(x[..., :4])
                y = pde[..., self.T_in:(self.T_in + self.T_out), :3]
                mask_y = (y[..., 0:1] == 0.0)
                y[..., 1:2][mask_y] = 0.0
                y[..., 2:3][mask_y] = 0.0
                mask_tensor = ~torch.isnan(y)
                y = torch.nan_to_num(y, nan=0.0)
            return x, y, mask_tensor
        pde_idx = idx // (self.nt - self.num_hist) # Markov / teacher forcing: only return one future step
        t_idx = idx % (self.nt - self.num_hist) + self.num_hist
        pde_path = self.data[pde_idx]
        # path_idx = os.path.join(path_root, str(pde_idx) + ".pt")
        # pde = torch.load(path_idx)
        pde = torch.load(pde_path)
        pde = pde.permute(1, 2, 0, 3)
        x = pde[..., (t_idx - self.num_hist):t_idx, :]
        mask = (x[..., 0:1] == 0.0)
        x[..., 1:2][mask] = 0.0
        x[..., 2:3][mask] = 0.0
        # mask_tensor = torch.isnan(x[..., :3])
        x[..., :3] = torch.nan_to_num(x[..., :3], nan=0.0)
        max_z = x[..., -1].max()
        z_value = max_z + 30.0
        x[..., -1] = torch.nan_to_num(x[..., -1], nan=z_value)
        x[..., -1] = torch.nn.functional.normalize(x[..., -1])
        x[..., 3:4] = self.log_transform(x[..., 3:4]) / 10.0
        # x[..., :4] = self.log_transform(x[..., :4])
        y = pde[..., t_idx, :3]
        mask_y = (y[..., 0:1] == 0.0)
        y[..., 1:2][mask_y] = 0.0
        y[..., 2:3][mask_y] = 0.0
        mask_tensor = ~torch.isnan(y)
        y = torch.nan_to_num(y, nan=0.0)
        if self.noise_std > 0:
            x += torch.randn(*x.shape, device=x.device) * self.noise_std
        x = torch.nan_to_num(x, nan=0.0)
        mask_tensor = torch.nan_to_num(mask_tensor, nan=0.0)
        return x, y, mask_tensor


In [5]:

def fine_tune_fc1_fc2(model, learning_rate):
    #self.fc1 and self.fc2
    for param in model.parameters():
        param.requires_grad = False

    for param in model.fc1.parameters():
        param.requires_grad = True
    for param in model.fc2.parameters():
        param.requires_grad = True
    # params_to_update = [p for p in model.parameters() if p.requires_grad]
    # if not params_to_update:
    #     raise ValueError("No parameters to update. Check if 'requires_grad' is set correctly.")
    #
    # optimizer = torch.optim.Adam(params_to_update, lr=learning_rate, weight_decay=args.lmbda)

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=args.lmbda)
    return optimizer

#self.fc1, self.fc2 and self.conv8
def fine_tune_fc1_fc2_conv8(model, learning_rate):
    for param in model.parameters():
        param.requires_grad = False

    for param in model.fc1.parameters():
        param.requires_grad = True
    for param in model.fc2.parameters():
        param.requires_grad = True
    for param in model.conv8.parameters():
        param.requires_grad = True

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=args.lmbda)
    return optimizer

#self.fc1, self.fc2, self.conv8 and self.conv7
def fine_tune_fc1_fc2_conv8_conv7(model, learning_rate):
    for param in model.parameters():
        param.requires_grad = False

    for param in model.fc1.parameters():
        param.requires_grad = True
    for param in model.fc2.parameters():
        param.requires_grad = True
    for param in model.conv8.parameters():
        param.requires_grad = True
    for param in model.conv7.parameters():
        param.requires_grad = True

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=args.lmbda)
    return optimizer

#self.fc1
def fine_tune_fc1(model, learning_rate):
    for param in model.parameters():
        param.requires_grad = False

    for param in model.fc1.parameters():
        param.requires_grad = True

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=args.lmbda)
    return optimizer

In [6]:
root = '/home/sirui/INNOMAUS/UrbanFloodCast/UrbanFloodCast/TL-DNO/save' 
path_model = os.path.join(root, 'model.pt')
#writer = SummaryWriter(root)

################################################################
# Model init
################################################################
if args.model_type in ["FNO2d", "FNO2d_aug"]:
    model = FNO2d(num_channels=num_channels, initial_step=initial_step, modes1=modes, modes2=modes, width=width,
                  grid_type=grid_type).to(device)
elif args.model_type in ["FNO3d", "FNO3d_aug"]:
    modes3 = time_modes if time_modes else modes
    model = FNO3d(num_channels=num_channels, initial_step=initial_step, modes1=modes, modes2=modes, modes3=modes3,
                  width=width, time=time1, time_pad=args.time_pad).to(device)
elif args.model_type == "DNO":
    model = DNO(num_channels=num_channels, width=10, initial_step=initial_step, pad=args.time_pad, factor=1).to(device)
elif args.model_type == "UNet3d":
    model = UNet3d(in_channels=initial_step * num_channels, out_channels=num_channels_y, init_features=32,
                   grid_type=grid_type, time=time1).to(device)
else:
    raise NotImplementedError("Model not recognized")

################################################################
# load data
# Input: DEM/Initial conditions/Rainfall/coords
################################################################
full_data = None # for superres
# SR dataset
dem_tif_path = '/home/sirui/INNOMAUS/wurzburg/dgm_wurzburg.tif'
man_path = '/home/sirui/INNOMAUS/wurzburg/friction_wurzburg_buf.tif'
Path_train = '/home/sirui/INNOMAUS/wurzburg/pt/train'
Path_valid = '/home/sirui/INNOMAUS/wurzburg/pt/val'
Path_test = '/home/sirui/INNOMAUS/wurzburg/pt/test'


In [7]:

train_data = flood_data(path_root=Path_train, strategy=args.strategy, T_in=T_in, T_out=T, std=args.noise_std)
ntrain = len(train_data)
print('ntrain', ntrain)
valid_data = flood_data(path_root=Path_valid, train=False, strategy=args.strategy, T_in=T_in, T_out=T)
nvalid = len(valid_data)
print('nvalid', nvalid)
test_data = flood_data(path_root=Path_test, train=False, strategy=args.strategy, T_in=T_in, T_out=T)
ntest = len(test_data)
print('ntest', ntest)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

################################################################
# training and evaluation
################################################################
pre_path = '/home/sirui/INNOMAUS/UrbanFloodCast/UrbanFloodCast/DNO/save/model5.pt'
checkpoint = torch.load(pre_path,map_location=torch.device('cpu'))
state_dict = checkpoint['state_dict']
ep=checkpoint['epoch']
model.load_state_dict(state_dict)
print("Load from checkpoint")
# for name, layer in model.named_modules():
#     print(f"Layer Name: {name}, Layer Object: {layer}")

complex_ct = sum(par.numel() * (1 + par.is_complex()) for par in model.parameters())
real_ct = sum(par.numel() for par in model.parameters())
if args.verbose:
    print(f"{args.model_type}; # Params: complex count {complex_ct}, real count: {real_ct}")
#writer.add_scalar("Parameters/Complex", complex_ct)
#writer.add_scalar("Parameters/Real", real_ct)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=args.lmbda)
#optimizer = fine_tune_fc1_fc2_conv8_conv7(model=model, learning_rate=learning_rate)
if args.step:
    assert args.step_size is not None, "step_size is None"
    assert scheduler_gamma is not None, "gamma is None"
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=scheduler_gamma)
else:
    num_training_steps = epochs * len(train_loader)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_training_steps)

lploss = LpLoss(size_average=False)

best_valid = float("inf")
best_epoch = 0
x_train, y_train, _ = next(iter(train_loader))
x_train = torch.nan_to_num(x_train, nan=0.0)
y_train = torch.nan_to_num(y_train, nan=0.0)
if np.isnan(x_train).any() or np.isnan(y_train).any():
    raise ValueError("NaN values found in training data")
x = x_train.to(device)
y = y_train.to(device)
x_valid, y_valid, _ = next(iter(valid_loader))
if args.verbose:
    print(f"{args.model_type}; Input shape: {x.shape}, Target shape: {y.shape}")
if args.strategy == "oneshot":
    assert x_train[0].shape == torch.Size([Sy, Sx, T, T_in, num_channels]), x_train[0].shape
    assert y_train[0].shape == torch.Size([Sy, Sx, T, num_channels_y]), y_train[0].shape
    assert x_valid[0].shape == torch.Size([Sy, Sx, T, T_in, num_channels]), x_valid[0].shape
    assert y_valid[0].shape == torch.Size([Sy, Sx, T, num_channels_y]), y_valid[0].shape
elif args.strategy == "markov":
    assert x_train[0].shape == torch.Size([Sy, Sx, 1, num_channels]), x_train[0].shape
    assert y_train[0].shape == torch.Size([Sy, Sx, num_channels_y]), y_train[0].shape
    assert x_valid[0].shape == torch.Size([Sy, Sx, 1, num_channels]), x_valid[0].shape
    assert y_valid[0].shape == torch.Size([Sy, Sx, T, num_channels_y]), y_valid[0].shape
else: # strategy == recurrent or teacher_forcing
    assert x_train[0].shape == torch.Size([Sy, Sx, T_in, num_channels]), x_train[0].shape
    assert x_valid[0].shape == torch.Size([Sy, Sx, T_in, num_channels]), x_valid[0].shape
    assert y_valid[0].shape == torch.Size([Sy, Sx, T, num_channels_y]), y_valid[0].shape
    if args.strategy == "recurrent":
        assert y_train[0].shape == torch.Size([Sy, Sx, T, num_channels_y]), y_train[0].shape
    else: # strategy == teacher_forcing
        assert y_train[0].shape == torch.Size([Sy, Sx, num_channels_y]), y_train[0].shape

model.eval()
start = default_timer()
if args.verbose:
    print("Training...")
step_ct = 0
train_times = []
eval_times = []
for ep in range(ep,epochs):
    model.train()
    t1 = default_timer()

    train_l2 = train_vort_l2 = train_pres_l2 = 0

    for xx, yy, mask in tqdm(train_loader, disable=not args.verbose):
        xx = torch.nan_to_num(xx, nan=0.0)
        yy = torch.nan_to_num(yy, nan=0.0)
        mask = torch.nan_to_num(mask, nan=0.0)
        loss = 0
        xx = xx.to(device)
        yy = yy.to(device)
        mask = mask.to(device)
        yy = yy * mask

        if args.strategy == "recurrent":
            for t in range(yy.shape[-2]):
                y = yy[..., t, :]
                mas = mask[..., t, :]
                im = model(xx)
                im = im * mas
                loss += lploss(im.reshape(len(im), -1, num_channels_y), y.reshape(len(y), -1, num_channels_y))
                xx = torch.cat((xx[..., 1:, :], im), dim=-2)
            loss /= yy.shape[-2]
        else:
            im = model(xx)
            im = im * mask
            if args.strategy == "oneshot":
                im = im.squeeze(-1)
            loss = lploss(im.reshape(len(im), -1, num_channels_y), yy.reshape(len(yy), -1, num_channels_y))

        train_l2 += loss.item()
        if swe:
            train_vort_l2 += lploss(im[..., VORT_IND].reshape(len(im), -1, 1), yy[..., VORT_IND].reshape(len(yy), -1, 1)).item()
            train_pres_l2 += lploss(im[..., PRES_IND].reshape(len(im), -1, 1), yy[..., PRES_IND].reshape(len(yy), -1, 1)).item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if not args.step:
            scheduler.step()
        #writer.add_scalar("Train/LR", scheduler.get_last_lr()[0], step_ct)
        step_ct += 1
    if args.step:
        scheduler.step()

    train_times.append(default_timer() - t1)

    # validation
    valid_l2 = valid_vort_l2 = valid_pres_l2 = 0
    valid_loss_by_channel = None
    with torch.no_grad():
        model.eval()
        model(xx)
        for xx, yy, mask in valid_loader:
            xx = torch.nan_to_num(xx, nan=0.0)
            yy = torch.nan_to_num(yy, nan=0.0)
            mask = torch.nan_to_num(mask, nan=0.0)
            xx = xx.to(device)
            yy = yy.to(device)
            mask = mask.to(device)
            yy = yy * mask

            pred = get_eval_pred(model=model, x=xx, strategy=args.strategy, T=T, times=eval_times).view(len(xx), Sy, Sx, T, num_channels_y)
            pred = pred * mask

            valid_l2 += lploss(pred.reshape(len(pred), -1, num_channels_y), yy.reshape(len(yy), -1, num_channels_y)).item()

    t2 = default_timer()
    if args.verbose:
        print(f"Ep: {ep}, time: {t2 - t1}, train: {train_l2 / ntrain}, valid: {valid_l2 / nvalid}")

    #writer.add_scalar("Train/Loss", train_l2 / ntrain, ep)
    #writer.add_scalar("Valid/Loss", valid_l2 / nvalid, ep)
    if ep % 20 == 0:
        path = os.path.join(root, f'model_epoch_{ep}.pt')
        torch.save({'epoch': ep, 'state_dict': model.state_dict()}, path)
    if valid_l2 < best_valid:
        best_epoch = ep
        best_valid = valid_l2
        # torch.save(model.state_dict(), path_model)
        state_dict = model.state_dict()
        torch.save({'epoch': best_epoch, 'state_dict': state_dict}, path_model)
    if args.early_stopping:
        if ep - best_epoch > args.early_stopping:
            break

stop = default_timer()
train_time = stop - start
train_times = torch.tensor(train_times).mean().item()
num_eval = len(eval_times)
eval_times = torch.tensor(eval_times).mean().item()
model.eval()
# test
##FNO
# model.load_state_dict(torch.load(path_model))
## Other models
checkpoint = torch.load(path_model)
state_dict = checkpoint['state_dict']
model.load_state_dict(state_dict)
model.eval()
test_l2 = test_vort_l2 = test_pres_l2 = test_nse = test_corr = test_csi_1 = test_csi_2 = test_csi_3 = 0
rotations_l2 = 0
reflections_l2 = 0
test_rt_l2 = 0
test_rf_l2 = 0
test_loss_by_channel = None
key = 0
i = 0
total_time = 0
sample_count = 0

with torch.no_grad():
    for xx, yy, mask in test_loader:
        xx = torch.nan_to_num(xx, nan=0.0)
        yy = torch.nan_to_num(yy, nan=0.0)
        mask = torch.nan_to_num(mask, nan=0.0)
        xx = xx.to(device)
        yy = yy.to(device)
        mask = mask.to(device)
        yy = yy * mask
        input_data = xx
        # print('xx', xx.shape)
        # print('yy', yy.shape)
        # Start
        start_time = time.time()
        pred = get_eval_pred(model=model, x=xx, strategy=args.strategy, T=T, times=[]).view(len(xx), Sy, Sx, T, num_channels_y)
        # End
        end_time = time.time()
        batch_time = end_time - start_time
        total_time += batch_time
        sample_count += len(xx)
        # print(f"Average prediction time per sample: {batch_time:.4f} seconds, lens of samples: {len(xx)}")

        pred = pred * mask
        # print('pred', pred.shape)
        test_l2 += lploss(pred.reshape(len(pred), -1, num_channels_y), yy.reshape(len(yy), -1, num_channels_y)).item()
        test_nse += nse(pred.reshape(len(pred), -1, num_channels_y), yy.reshape(len(yy), -1, num_channels_y)).item()
        test_corr += corr(pred.reshape(len(pred), -1, num_channels_y), yy.reshape(len(yy), -1, num_channels_y)).item()
        test_csi_1 += critical_success_index(pred[..., 0:1].reshape(len(pred), -1, 1),
                                             yy[..., 0:1].reshape(len(yy), -1, 1), 0.01).item()
        test_csi_2 += critical_success_index(pred[..., 0:1].reshape(len(pred), -1, 1),
                                             yy[..., 0:1].reshape(len(yy), -1, 1), 0.1).item()
        test_csi_3 += critical_success_index(pred[..., 0:1].reshape(len(pred), -1, 1),
                                             yy[..., 0:1].reshape(len(yy), -1, 1), 0.5).item()
        
print('sample_count', sample_count)
print('ntest', ntest)
average_time_per_sample = total_time / sample_count if sample_count > 0 else 0
test_time_l2 = test_space_l2 = ntest_super = test_int_space_l2 = test_int_time_l2 = None

print(f"Average prediction time per sample: {average_time_per_sample:.4f} seconds")
print(f"{args.model_type} done training; \nTest: {test_l2 / ntest}, Test_nse: {test_nse / ntest}, Test_corr: {test_corr / ntest}, Test_csi_1: {test_csi_1 / ntest}, Test_csi_2: {test_csi_2 / ntest}, Test_csi_3: {test_csi_3 / ntest}")
summary = f"Args: {str(args)}" \
          f"\nParameters: {complex_ct}" \
          f"\nTrain time: {train_time}" \
          f"\nMean epoch time: {train_times}" \
          f"\nMean inference time: {eval_times}" \
          f"\nNum inferences: {num_eval}" \
          f"\nTrain: {train_l2 / ntrain}" \
          f"\nValid: {valid_l2 / nvalid}" \
          f"\nTest: {test_l2 / ntest}" \
          f"\nTest_nse: {test_nse/ntest}" \
          f"\nTest_corr: {test_corr/ntest}" \
          f"\nTest_csi_1: {test_csi_1/ntest}" \
          f"\nTest_csi_2: {test_csi_2/ntest}" \
          f"\nTest_csi_3: {test_csi_3/ntest}" \
          f"\nSuper Space Test: {test_space_l2}" \
          f"\nSuper Space Interpolation Test: {test_int_space_l2}" \
          f"\nSuper S: {S_super}" \
          f"\nSuper Time Test: {test_time_l2}" \
          f"\nSuper Time Interpolation Test: {test_int_time_l2}" \
          f"\nSuper T: {T_super}" \
          f"\nBest Valid: {best_valid / nvalid}" \
          f"\nEpochs trained: {ep}"
if swe:
    summary += f"\nVorticity Test: {test_vort_l2 / ntest}" \
               f"\nPressure Test: {test_pres_l2 / ntest}"
txt = "results"
if args.txt_suffix:
    txt += f"_{args.txt_suffix}"
txt += ".txt"

#with open(os.path.join(root, txt), 'w') as f:
#    f.write(summary)
#writer.flush()
#writer.close()

Number of .pt files: 16
ntrain 16
Number of .pt files: 1
nvalid 1
Number of .pt files: 1
ntest 1
Load from checkpoint
DNO; # Params: complex count 8937637, real count: 4470437
DNO; Input shape: torch.Size([1, 278, 227, 120, 1, 5]), Target shape: torch.Size([1, 278, 227, 120, 3])
Training...


100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [15:08<00:00, 56.80s/it]


Ep: 8, time: 944.0196429498028, train: 1.1330672316253185, valid: 1.0162140130996704


100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [14:09<00:00, 53.12s/it]


Ep: 9, time: 887.3793207928538, train: 0.972096111625433, valid: 0.9701599478721619


...

Ep: 497, time: 836.768689227989, train: 0.1853294325992465, valid: 0.1963752657175064


100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [13:06<00:00, 49.16s/it]


Ep: 498, time: 820.7957477851305, train: 0.18533393368124962, valid: 0.19645161926746368


100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [12:47<00:00, 47.95s/it]


Ep: 499, time: 800.9583742560353, train: 0.18532124999910593, valid: 0.19639728963375092
sample_count 1
ntest 1
Average prediction time per sample: 16.5559 seconds
DNO done training; 
Test: 0.24351851642131805, Test_nse: 0.9402858018875122, Test_corr: 0.9699470400810242, Test_csi_1: 0.5142586827278137, Test_csi_2: 0.7684683203697205, Test_csi_3: 0.7935432195663452
